In [2]:
from agents.collection_agent import CollectionAgent
from agents.source_material import SourceMaterial
from agents.test_question import TestQuestions, Question
import os
from glob import glob
import gradio as gr
import pandas as pd
from openai import OpenAI
openai = OpenAI()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
files = glob("/Users/admin/Projects/MathTestCreator/Documents/*")

In [59]:
collectionAgent = CollectionAgent()
collectionAgent.create_source_material(files)
collectionAgent.setup()

file added
initial collection set


Collection(name=math_documents)

In [68]:
result = collectionAgent.create_formatted_test( 'math', 11, 3, 'trigonometry', 'short answer, fill in the blank')

In [70]:
result.questions

[Question(number=1, question='In the equation d² −2kd cos θ + k² − r² = 0, identify the term that represents the product of the two roots of this equation.'),
 Question(number=2, question='Given the quadratic equation d² −2kd cos θ + k² − r² = 0, fill in the blank: The expression that is independent of the angle θ is _____.'),
 Question(number=3, question='In the context of the equation d² −2kd cos θ + k² − r² = 0, fill in the blank: The value of k² − d² represents the product of the roots, which is _____.')]

In [91]:
material.metadata['page']

0

In [45]:
type(1)

int

In [26]:
css_string =".editable-table th {font-style: italic;} .editable-table td {font-style: italic;cursor: pointer;} .locked-table th {font-weight: bold;} .locked-table td {font-weight: normal;cursor: not-allowed; border: 0}"

In [16]:
css_string = """.green-button {background-color: #32de84} .green-button:hover {color:#32de84}  .error {color : #cc0000, font-size:24 }
<style> .custom-padding {
        padding: 20px;
    }
</style>"""


In [76]:
ui.close()
collectionAgent = CollectionAgent()
def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history
def stream_test(result):
        test_markdown = ''
        for item in result.questions:
            test_markdown += (f"{item.number}. {item.question} \n\n")
            yield test_markdown
def file_test(result):
     with ('test_file.txt', 'wb') as f:
          for item in result.questions:
            f.write(f"{item.number}. {item.question} \n\n")
    f.close()

              
def docdata_to_text(material):
    # for material in collectionAgent.materials:
    if material.supported:
            yield f"### Document Successfully Uploaded\n\n {repr(material)}"
    else:
        yield '### Unsupported Document Type' 
def create_test_output(json):
    return json
def doc_to_df(doc_df, new_material):
    try:
        if new_material:
            if new_material.supported:
                index_value = len(doc_df) if doc_df.loc[0]["Name"] else 0
                doc_df.loc[index_value] = [index_value+1,
                                           new_material.name,
                                           0,
                                           int(new_material.length),
                                           new_material.length_type 
                                        ]
            return doc_df,  gr.update(visible=True), gr.update(visible=True)
    except Exception as e:
        print('noooooo')
    return doc_df,  gr.update(visible=False), gr.update(visible=False)
with gr.Blocks(css=css_string, theme=gr.themes.Ocean() ,title="Custom Math Test Creator") as ui:
    questions = gr.State()
    new_doc = gr.State()
    #Set Parameters
    gr.Markdown('## Custom  Test Creator')

    with gr.Tab('Material'):                
        with gr.Row():
                gr.Markdown("#### Upload source materials for your test:")
        with gr.Row():
                with gr.Column():
                    file_uploader = gr.File(file_count="multiple",  height=175)
                with gr.Column():
                    # gr.Markdown("### Here are the documents you have uploaded")
                    doc_data = gr.Markdown(label="Document Data", height=175)
        with gr.Row():
            doc_markdown = gr.Markdown("### Uploaded Documents", visible=False)

        with gr.Row():
            doc_df = gr.Dataframe(
                headers = ["Material", "Name", "Start", "End", "Unit"],
                interactive=True,
                column_widths=[1,3,1,1,2],
                datatype=["number", "str","number","number","str" ],
                col_count=(5, "fixed"),
                visible = False,
                wrap = True,
                elem_classes = "editable-table",
            )

        with gr.Row():
            with gr.Column():
                no_rag = gr.Button("Continue without materials (not recommended)")
            with gr.Column():
                create_rag = gr.Button("Material Upload Completed", visible=False, elem_classes="green-button")
        with gr.Row():
            error_messages = gr.Markdown(elem_classes="error")
        with gr.Row():
            success_messages = gr.Markdown("###")

    with gr.Tab("Test Creator"): 
        gr.Markdown("## Welcome to your test creator")
        gr.Markdown(f"### Create a test based on your material uploads:")
        document_markdown = gr.Markdown()
    
        with gr.Row(): 
            subject = gr.Textbox(label = "Subject", value='Math')
            grade_level = gr.Dropdown(label="Grade Level", choices=[6,7,8,9,10,11,12,"College", "University"])
        with gr.Row():
            with gr.Column(scale=2):
                question_number = gr.Textbox(2,label = "Number of questions")
                topics = gr.Textbox('trig',label = "Topics")
                question_type = gr.Dropdown( label="Types of questions", choices=['Mulitple Choice', 'Essay', 'True/False', 'Short Answer', 'File in the Blank'],multiselect=True)
                create_test = gr.Button("Create a test")
                test_feedback = gr.Textbox(label="Give feedback on the test")
            with gr.Column(scale=3):
                test_markdown = gr.Markdown(label="Test:", min_height=300)
           
        # related = gr.Textbox('related')   
        download_test = gr.DownloadButton(label='Download', value='test_file.txt')
    
        # with gr.Row():
        #     with gr.Column():
        #         chatbot = gr.Chatbot(type='messages')
                # chat_entry = gr.Textbox() 
            # with gr.Column(scale=2):
                #  test = gr.Markdown("## Test")          
    def check_integer_values(doc_df):
        starts = [start for start in doc_df["Start"]]
        ends=  [end for end in doc_df["End"]]
        digits = [x for x in starts + ends if not str(x).isdigit()]
        if len(digits) > 0:
            return f"starts and ends must be integers", gr.update(visible=False)
        if doc_df.loc[0]["Name"]:
            return "", gr.update(visible=True)
        else:
            return "", gr.update(visible=False)
        
        
    file_uploader.change(collectionAgent.create_source_material, inputs=[file_uploader], outputs=[new_doc])
    new_doc.change(docdata_to_text, inputs=[new_doc], outputs=[doc_data]).then(doc_to_df, inputs=[doc_df, new_doc], outputs=[doc_df, doc_df, create_rag])
    # .then(populate_material,inputs=doc_df, outputs=document_markdown) 
    # chat_entry.submit(do_entry, inputs=[chat_entry, chatbot], outputs=[chat_entry, chatbot]).then(
    #     chat_gpt, inputs=[chatbot,question_number, topics, question_type, doc_df], outputs=[chatbot]).then(collectionAgent.create_collection)
    # test_create.click(collectionAgent.setup).then(collectionAgent.gpt_4o_mini_rag, inputs=[chatbot], outputs=[output])
    doc_df.change(check_integer_values, inputs=[doc_df], outputs=[error_messages,create_rag])
    
    create_rag.click(collectionAgent.materials_rag, inputs=[doc_df], outputs=[success_messages]).then(collectionAgent.setup)
    create_test.click(collectionAgent.create_formatted_test, inputs=[subject, grade_level, question_number, topics,question_type], outputs=[questions]).then(stream_test, inputs=[questions], outputs=[test_markdown])
    test_feedback.submit(collectionAgent.create_formatted_test, inputs=[subject, grade_level, question_number, topics,question_type, test_feedback], outputs=[questions]).then(stream_test, inputs=[questions], outputs=[test_markdown]).then(file_test, inputs=[questions])

ui.launch(inbrowser=True)
        

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/admin/anaconda3/envs/testg/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 1 of document /private/var/folders/sn/z9t3m2s11ml9z7s806xf76_m0000gn/T/gradio/e044d7a4cf955c7799a4f2904f5ff1f567df3699e3c3a6a3cea772b79aa02d96/further.pdf
  warnings.warn(


file added
initial collection set


In [87]:
ui.close()


Closing server running on port: 7888
